Data Cleaning and preprocessing has been done using R. This file contains the different models used

In [1]:
#Import all the required libraries for the project
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('O_Final_LoanDefault.csv')
df


,Unnamed: 0,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,loan_amount,...,total_units,income,credit_type,Credit_Score,co.applicant_credit_type,age,submission_of_application,LTV,dtir1,Status
0,1.0,0,0,1,0,0,0,0,0,116500,...,1,1740,0,758,1,1,1,98.728814,45,1
1,2.0,0,0,1,1,0,0,0,1,206500,...,1,4980,1,552,0,4,1,90.680473,51,1
2,2.1,0,0,1,1,0,0,0,1,206500,...,1,4980,1,552,0,4,1,90.680473,51,1
3,2.2,0,0,1,1,0,0,0,1,206500,...,1,4980,1,552,0,4,1,90.680473,51,1
4,3.0,0,0,0,0,0,0,0,0,406500,...,1,9480,0,834,1,2,1,80.019685,46,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192631,148666.0,0,2,1,0,2,0,0,0,436500,...,1,7860,3,659,0,4,1,71.792760,48,0
192632,148667.0,0,0,1,0,0,0,0,0,586500,...,4,7140,3,569,1,1,0,74.428930,15,0
192633,148668.0,0,0,1,0,3,0,0,0,446500,...,1,6900,3,702,0,3,0,61.332420,49,0
192634,148669.0,0,1,1,0,3,0,0,0,196500,...,1,7140,0,737,0,4,1,70.683450,29,0


In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,28:29], test_size=0.2, random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(154108, 28)
(38528, 28)
(154108, 1)
(38528, 1)


In [4]:
print(y_train)

        Status
20097        0
74856        0
131689       0
12229        0
173480       0
...        ...
119879       0
103694       1
131932       0
146867       0
121958       0

[154108 rows x 1 columns]


Model 1: Random forest with Randomized Search CV

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestClassifier(criterion='gini', random_state=0)
maxFeatures = range(1,df.shape[1]-1)
param_dist = dict(max_features=maxFeatures)
rand = RandomizedSearchCV(rf, param_dist, cv=10, scoring='accuracy', n_iter = 6, n_jobs = 3, random_state=10)
rand.fit(x_train,y_train)
mean_scores = [result.mean_validation_score for result in rand.grid_scores_]
#print('Best Accuracy = '+str(rand.best_score_))
print(rand.best_estimator_)

KeyboardInterrupt: ignored

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
randomForest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=6, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)
randomForest.fit(x_train,y_train)
rfPredict = randomForest.predict(x_test)
rfPredictproba = randomForest.predict_proba(x_test)[:,1] #for ROC curve
rfAccuracy = accuracy_score(y_test,rfPredict)
roc_score = metrics.roc_auc_score(y_test,rfPredict)
print(rfAccuracy)
print(classification_report(y_test, rfPredict))

0.9512043189368771
              precision    recall  f1-score   support

           0       0.94      0.97      0.96     22554
           1       0.96      0.92      0.94     15974

    accuracy                           0.95     38528
   macro avg       0.95      0.95      0.95     38528
weighted avg       0.95      0.95      0.95     38528



Using Bagging along with random forest

In [13]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed, shuffle = True)
num_trees = 100
model = BaggingClassifier(base_estimator=randomForest, n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, df.iloc[:,:-1].values, df.iloc[:,-1].values, cv=kfold)
print(results.mean())

KeyboardInterrupt: ignored

Model 2: Logistic Regression 

In [14]:
from sklearn import linear_model
clf_LR = linear_model.LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
clf_LR.fit(x_train,y_train)
LR_Predict = clf_LR.predict_proba(x_test)[:,1]
LR_Predict_bin = clf_LR.predict(x_test)
LR_Accuracy = accuracy_score(y_test, LR_Predict_bin)
print(classification_report(y_test, LR_Predict_bin))

              precision    recall  f1-score   support

           0       0.59      1.00      0.74     22554
           1       1.00      0.00      0.00     15974

    accuracy                           0.59     38528
   macro avg       0.79      0.50      0.37     38528
weighted avg       0.76      0.59      0.43     38528



To conclude, it seems that logistic regression is underfitting the dataset while Random forest overfits the dataset.

Model 3: AdaBoost Classifier

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
Ada_clf = AdaBoostClassifier(n_estimators=50)
Ada_clf.fit(x_train, y_train)
y1 = Ada_clf.predict(x_test)
accuracy = accuracy_score(y1, y_test)
scores = cross_val_score(Ada_clf, df.iloc[:,:-1], df.iloc[:,-1])
print(accuracy)
print(y1)
print(classification_report(y_test, y1))

0.8018843438538206
[0 0 0 ... 1 1 1]
              precision    recall  f1-score   support

           0       0.78      0.91      0.84     22554
           1       0.84      0.64      0.73     15974

    accuracy                           0.80     38528
   macro avg       0.81      0.78      0.79     38528
weighted avg       0.81      0.80      0.80     38528

